# Preparando datos para armar un motor de trading

En la industria del trading algorítmico, las etiquetas para predicción no están del todo claras en un comienzo. Además, el hecho de estar trabajando con series de tiempo complica el trabajo. Vamos a dejar un poco de lado este hecho y vamos a "jugar" un poco con los datos.

Esta vez vamos a trabajar con datos de acciones argentinas y vamos a ver qué podemos encontrar. Una acción o "stock", como se la conoce en inglés, es un título público que emiten las empresas para obtener inversión para sus proyectos. Es diferente a los bonos, ya que no tiene flujos de fondos a futuro, si no que representa la posibilidad de ser parte de la sociedad. Al comprar acciones de una empresa, estamos siendo socios de la misma y el valor de nuestra participación puede subir y bajar con las buenas y las malas que pase la empresa. Como podemos adivinar, este mercado es aún más volátil que el de los bonos (algo que en Argentina no es poco decir...).

En los mercados no solamente existen los precios de las acciones, si no que hay índices que trackean el rendimiento en general de las mismas. Para Argentina, el índice principal es el Merval (Qué es? [shameless plug](https://blog.invera.com.ar/qu%C3%A9-es-el-merval-6ce8df474e8c)).

Comencemos como en el práctico anterior (hace tantos años...), descargando los datos con los cuales vamos a trabajar desde la página web de Rava, que ahora nos deja invertir en la bolsa armando una cuenta con ellos (qué bien!), siempre y cuando tengamos al menos 300.000 pesos para invertir (qué mal).

In [1]:
### Needed imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import optimize
%matplotlib inline
import sys
from io import StringIO

### Lista de acciones argentinas ###
stocks_list = [
    'AGRO', 'ALUA', 'APBR', 'AUSO', 'BBAR', 'BHIP', 'BMA', 'BOLT', 'BPAT', 'BRIO', 'BYMA', 'CARC'
    'CAPU', 'CELU', 'CEPU', 'CGPA2', 'COME', 'CRES', 'CTIO', 'CVH', 'DGCU2', 'EDN', 'GCLA', 'GGAL', 'HAVA',
    'INDU', 'IRSA', 'LEDE', 'LOMA', 'LONG', 'METR', 'MIRG', 'MOLA', 'MOLI', 'MORI', 'OEST', 'PAMP',
    'PATY', 'PGR', 'SAMI', 'SUPV', 'TECO2', 'TGLT', 'TGNO4', 'TGSU2', 'TRAN', 'TS', 'TXAR', 'VALO', 'YPFD',
    ### Agregamos el índice de acciones argentinas: Merval
    'MERVAL',
]

In [3]:
### Obtengamos los datos y los guardemos por si las moscas...
for stock in stocks_list:
    url = 'http://www.ravabursatil.com/v2/empresas/precioshistoricos.php?e={}&csv=1'.format(stock)
    req = requests.get(url, allow_redirects=True)
    df = pd.read_csv(StringIO(req.content.decode("utf-8")))
    name = './stocks/{}.csv'.format(stock)
    df.to_csv(name)

# Hagamos Análisis Técnico!

El *análisis técnico* es el "arte" (no me atrevo a llamar a esto ciencia) de predecir los movimientos futuros de los activos financieros en base a su comportamiento en el pasado. Como con la predicción del clima, puede no ser preciso, pero quizás dé algunas ideas de cómo se pueden llegar a mover ciertos activos y se puede tomar decisiones de inversión a partir de esto.

Vamos a tener ciertas suposiciones claves para hacer con respecto a esto:
- **Alta liquidez**: Se asume que las acciones con las que se trabaja tienen un alto volumen de negociación. Si una acción se negocia mucho, es mucho más difícil manipular su precio, por lo que seguiría un curso "de manual", si es que se encuentra ese manual...
- **No hay cambios artificiales de precios**: Las acciones pagan dividendos (como el cupón de un bono) o, en algunos casos, pueden ser divididas (mediante una acción que se denomina "split"). Estas acciones pueden cambiar de manera artificial el precio de una acción. Para poder tener esto en cuenta, es necesario tener el precio de una acción "ajustado" por sus movimientos artificiales. Por suerte, la data de RAVA ya viene con esto solucionado.
- **No hay "cisnes negros"**: El análisis técnico no puede tener en cuenta noticias externas que puedan afectar al desempeño de una acción de manera sorpresiva, como por ejemplo la muerte de su CEO o un ataque terrorista en el que se viera afectada.

En resumen, el análisis técnico utiliza la información capturada en el precio de un activo para interpretar qué es lo que el mercado dice, para formar una visión del futuro.

Claramente, esto puede sonar a chamuyo, pero puede ser algo de qué agarrarse cuando no se tiene mucha más información con la cual trabajar, ya que ha mostrado ser de utilidad en mercados muy grandes, como en EE.UU.

### Será posible aplicarlo en el mercado argentino? ###

Otra cosa a tener en cuenta: Los datos históricos de mercado generan **series de tiempo**, que son un fenómeno un poco lejano a los datos estáticos. En este práctico vamos a probar las herramientas aprendidas en la teoría y ver si pueden tener incidencia a la hora de trabajar con estas series de tiempo.